In [51]:
import pandas
from sklearn.linear_model import MultiTaskElasticNetCV, ElasticNetCV
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import normalize
from sklearn.ensemble import BaggingRegressor
from scipy.stats import pearsonr
from matplotlib import use; use('Agg')
import matplotlib.pyplot as plt 
%matplotlib inline


In [52]:
!ls benchmarks/*sc
!wc -l benchmarks/*sc 

benchmarks/clean_repack_score.sc benchmarks/original.sc           benchmarks/whole60.sc
benchmarks/detect.sc             benchmarks/whole.sc
benchmarks/nstruct100.sc         benchmarks/whole10.sc
     123 benchmarks/clean_repack_score.sc
     123 benchmarks/detect.sc
   10398 benchmarks/nstruct100.sc
   10398 benchmarks/original.sc
     119 benchmarks/whole.sc
    1220 benchmarks/whole10.sc
    9960 benchmarks/whole60.sc
   32341 total


# Using the structural metrics calculated by the enzyme design mover 

In [ ]:
%%timeit -r1 -n1

# this is the new benchmark script 

def low_10( df ):
    return df.sort_values( by='total_score' ).head( 10 )

for scorefile in [ 'benchmarks/original.sc', 'benchmarks/whole10.sc', 'benchmarks/whole60.sc', 
                   'benchmarks/detect.sc', 'benchmarks/nstruct100.sc' ]:
    
    print '------------------------------------------------------------------------'
    print 'Results for scorefile "{}"'.format( scorefile )
    print '------------------------------------------------------------------------'
    
    df = pandas.read_csv( scorefile, sep=r'\s+' ).dropna()
    df.description = df.description.str[:-5]
    df = df.groupby( 'description' ).apply( low_10 )
    df.set_index( 'description', inplace=True )
    #print df.head()

    #drop_list = [ u'all_cst', u'tot_seq_recovery', u'SR_1', u'SR_1_all_cst', u'SR_2', u'SR_2_all_cst', 
    #              u'SR_3', u'SR_3_all_cst', u'SR_4', u'SR_4_all_cst', u'SR_5', u'SR_5_all_cst' ]
    #cols = [ i for i in drop_list if i in df.columns ]
    #df.drop?
    #print 'Dropping columns ', cols

    train_set = pandas.read_csv( '../data/train_set.csv' )
    train_set.set_index( 'mutant', inplace=True )

    fig, ax = plt.subplots( ncols=3, nrows=1, figsize=(10,3) )
    constants = [ 'kcat', '1/km', 'kcat/km' ]


    for const_index, constant in enumerate( constants ): 

        X = df.join( train_set ).dropna()
        y = X[ constant ].values
        X = X.ix[ :,:'expressed' ].values

        net = ElasticNetCV( normalize=True, selection='random' )

        params_grid = {
            'cv': [ 10 ], 
            'l1_ratio': [ 0.001, 0.01, 0.1, 0.5, 0.9 ], 
        }
        
        print 'Training on constant "{}" ...'.format( constant ) , 
        grid = GridSearchCV( net, params_grid )
        bag = BaggingRegressor( base_estimator=grid, n_estimators=1000, bootstrap_features=True )
        bag.fit( X, y )

        print 'done'
        print 'Calculating predictions for "{}" ...'.format( constant ),
        preds = bag.predict( X )

        pcc = pearsonr( preds, y )
        score = bag.score( X, y )
        #params = bag.get_params()
        
        print 'done', 
        #print '------------------------------------------------------------------------'
        print '\tPCC: {:.2f}, model score: {:.3f}'.format( pcc[0], score )
        ax[ const_index ].scatter( preds, y, alpha=0.3, marker='.', color='magenta' )
        ax[ const_index ].set_xlabel( 'Predicted {}'.format( constant ) )
        ax[ const_index ].set_ylabel( 'Actual' )
        plt.tight_layout()
        
    fig.suptitle( scorefile )
    fig.tight_layout()
    fig.show()


------------------------------------------------------------------------
Results for scorefile "benchmarks/original.sc"
------------------------------------------------------------------------
Training on constant "kcat" ...

# Using a features reporter to extract structural features 

In [ ]:
import sqlite3
from sklearn.cross_validation import cross_val_predict
from sklearn.ensemble import BaggingRegressor

con = sqlite3.connect( 'features.db3' )
for i in [ 'interfaces', 'interface_sides' ]:
    f = pandas.read_sql_query( 'select * from {}'.format( i ), con, index_col='struct_id' )

names = pandas.read_sql_query( 'select * from structures', con, index_col='struct_id' )
feats = f.join( names ).dropna()
#print f.head()
#print names.head()
#print feats.head()

feats['shlag'] = feats.tag.str[:-10]
feats.set_index( 'shlag', inplace=True )

train_set = pandas.read_csv( '../data/train_set.csv' )
train_set.set_index( 'mutant', inplace=True )

J = feats.join( train_set ).dropna()

y = J.ix[:,'kcat':]
X = J.ix[:,'dSASA':'batch_id'] 

net = MultiTaskElasticNetCV( )
params_grid = { 'l1_ratio': [ 0.001, 0.01, 0.1, 0.5 ], 'cv': [ 10 ] }
grid = GridSearchCV( net, params_grid )
grid.fit( X, y )

print 'Model score:', grid.score( X, y )
print 'Found params:', grid.best_params_
preds = [ grid.predict( X.iloc[i].reshape( 1, -1 ) ) for i in range( len( X ) ) ]
actuals = [ y.iloc[i] for i in range( len( X ) ) ]
#print 'Pearson {}'.format( pearsonr( preds, actuals ) )

fig, ax = plt.subplots( ncols=3, nrows=1, figsize=(10,3) )
for i in range( 3 ):
    xvalues = [ preds[n][0][i] for n in range( len( preds ) ) ]
    yvalues = [ actuals[n][i] for n in range( len( actuals ) ) ]
    ax[i].scatter( yvalues, xvalues, alpha=0.4, marker='.' )
    ax[i].set_xlabel( 'Actual' )
    ax[i].set_ylabel( 'Predicted' )
    ax[i].set_title( '{}\nPCC={:.2}'.format( y.columns[i], pearsonr( xvalues, yvalues )[0] ) )
    plt.tight_layout()

#bag = BaggingRegressor( base_estimator = grid )
#bag.fit( X, y )

#net = ElasticNetCV(cv=strat)

#preds = cross_val_predict( bag, X, y=y, cv=strat )
#net.fit( X, y )
#constants = ['kcat', '1/km', 'kcat/km']
#preds = pandas.DataFrame( preds, columns=constants )
#actuals = pandas.DataFrame( y, columns=constants )
#for c in constants:
#    plt.scatter( preds[c], actuals[c] )
#    plt.show()